In [7]:
import anywidget
import traitlets
import pandas as pd
from ipyleaflet import Map, DrawControl
from ipywidgets import VBox, Output



In [8]:
class MapWidget(anywidget.AnyWidget):
    data = traitlets.Any().tag(sync=True)
    def __init__(self, center=(40, -100), zoom=4, **kwargs):
        super().__init__(**kwargs)
        self.df = pd.DataFrame(columns=['southwest', 'northeast'])
        self.map = Map(center=center, zoom=zoom)
        self.draw_control = DrawControl(
            rectangle={"shapeOptions": {"color": "#0000FF"}}, polyline={}, polygon={}, circle={}, circlemarker={}, marker={})
        self.map.add_control(self.draw_control)
        self.draw_control.on_draw(self.handle_draw)
        self.out = Output()
        self.children = [self.map, self.out]
        
    def handle_draw(self, target, action, geo_json):
        if action == 'created' and geo_json['geometry']['type'] == 'Polygon':
            coords = geo_json['geometry']['coordinates'][0]
            lats = [c[1] for c in coords]
            lons = [c[0] for c in coords]
            sw = (min(lats), min(lons))
            ne = (max(lats), max(lons))
            self.df.loc[len(self.df)] = [sw, ne]
            self.data = self.df.to_dict(orient="records")
            with self.out:
                print(f"Rectangle added: SW={sw}, NE={ne}")
                print(self.df)

widget = MapWidget()
display(widget)

MapWidget()